In [0]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-05-13 02:40:12--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2020-05-13 02:40:12 (8.62 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [0]:
from pyspark import SparkFiles
from pyspark.sql import SQLContext, Row


# url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt"
# spark.sparkContext.addFile(url)

sc = spark.sparkContext
file_url = sc.textFile("https://s3.amazonaws.com/amazon-reviews-pds/tsv/index.txt")
reader = file_url.map(lambda x: x.split('\t')).map(lambda t: (t[0],t[1],int(t[2]),t[3]))

In [44]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy", header=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...|2015-08-31 00:00:00|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...| 

In [45]:
#count of rows
df.count()

9002021

In [46]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [0]:
mode = 'append'
jdbc_url = 'jdbc:postgresql://mypostgressdb.c94sats96kyu.us-west-1.rds.amazonaws.com:5432/my_data_class_db'
config = {
    "user": "root",
    "password": "password",
    "driver": "org.postgresql.Driver"
}

In [50]:
review_id_table = df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")
review_id_table.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543|2015-08-31 00:00:00|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650|2015-08-31 00:00:00|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154|2015-08-31 00:00:00|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704|2015-08-31 00:00:00|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300|2015-08-31 00:00:00|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488|2015-08-31 00:00:00|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554|2015-08-31 00:00:00|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702|2015-08-31 00:00:00|
|R2L15IS24CX0LI|   12738196|B00HVORET8|     389677711|2015-08-31 00:00:00|
|R1DJ8976WPWVZU|   15867807|B00HX3G6J6|     299654876|2015-08-31 00:00:00|
|R3MRWNNR8CBTB7|    19722

In [0]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [52]:
products = df.select("product_id", "product_title")
products.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00YL0EKWE|LG G4 Case Hard T...|
|B00XK95RPQ|Selfie Stick Fibl...|
|B00SXRXUKO|Tribe AB40 Water ...|
|B009V5X1CE|RAVPower® Element...|
|B00D93OVF0|Fosmon Micro USB ...|
|B00XVGJMDQ|iPhone 6 Case, Vo...|
|B00KQW1X1C|Nokia Lumia 630 R...|
|B00IP1MQNK|Lumsing 10400mah ...|
|B00HVORET8|iPhone 5S Battery...|
|B00HX3G6J6|HTC One M8 Screen...|
|B00U4NATNQ|S6 Case - Bear Mo...|
|B00SZEFDH8|BLU Studio X, Unl...|
|B00JRJUL9U|EZOPower 5-Port U...|
|B00KQ4T0HE|iPhone 6S Case &i...|
|B00M0YWKPM| iPhone 6s Plus Case|
|B00KDZEE68|zBoost ZB575-A TR...|
|B00BJN45GM|OtterBox Defender...|
|B00SA86SXW|Aduro PowerUP 30W...|
|B00Q3I68TU|LilGadgets Connec...|
|B00TN4J1TA|Anker Aluminum Mu...|
+----------+--------------------+
only showing top 20 rows



In [0]:
products.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)